# Train a TAN on the cifar10 subset

In [1]:
import os

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import howtotan
tan_nll = howtotan.get_tan_nll

print( "The Tensorflow version is " + tf.__version__ )

The Tensorflow version is 1.10.1


In [2]:
# Read in the data
def read_cifar10( filename_queue ):
    # Used cifar10_inputs.py as a guide
    
    height = 32
    width  = 32
    depth  = 3
    
    image_bytes = height * width * depth
    label_bytes = 1
    
    reader   = tf.FixedLengthRecordReader( record_bytes=( image_bytes + label_bytes ) )
    _, value = reader.read( filename_queue )
    
    data = tf.decode_raw( value, tf.uint8 )
    
    labels = tf.cast( 
        tf.strided_slice( data, [0], [label_bytes] ), tf.int32 )
    data = tf.transpose( 
        tf.reshape( 
            tf.strided_slice( data, [label_bytes], [label_bytes + image_bytes] ),
            [depth, height, width] ), 
        [1, 2, 0] )
    
    return data, labels

def build_batch( data, label, batch_size=32, shuffle=True, min_queue_ex=None):
    # WAG at min_queue_ex - not sure I really understand what this is for...
    
    if ( min_queue_ex is None ):
        min_queue_ex = 4 * batch_size
    
    num_pre_threads = 16
    if ( shuffle ):
        data_batch, label_batch = tf.train.shuffle_batch( 
            [data, label], 
            batch_size=batch_size, 
            num_threads=num_pre_threads, 
            capacity=min_queue_ex + batch_size, 
            min_after_dequeue=min_queue_ex
        )
    else:
        data_batch, label_batch = tf.train.batch( 
            [data, label], 
            batch_size=batch_size, 
            num_threads=num_pre_threads, 
            capacity=min_queue_ex + batch_size
        )
    
    # Visualize the training data
    tf.summary.image( 'images', data_batch )
    
    return data_batch, tf.reshape( label_batch, [batch_size] )

def creader( data_dir, batch_size, num_epochs ):
    filenames = [os.path.join( data_dir, "data_batch_{}.bin".format( ii ) ) 
                 for ii in xrange( 1, 6 )]
    
    for ff in filenames:
        if not tf.gfile.Exists( ff ):
            raise ValueError( "Could not find file {}".format( ff ) )
    
    # It would be better not redefine this multiple times
    height = 32
    width  = 32
    depth  = 3
    
    with tf.name_scope( 'read' ):
        filename_queue = tf.train.string_input_producer( filenames, num_epochs=num_epochs )
        
        data, labels = read_cifar10( filename_queue )
        data = tf.cast( data, tf.float32 ) / 255 # why do this here and not above?
        # Data is between zero and 1
        
        # Zero mean, unit variance
        # data = tf.image.per_image_standardization( data )
        
        data.set_shape( [height, width, depth])
        labels.set_shape( [1] )
        # What does set_shape do?
        
        # Return the data and the labels in the batch
        return build_batch( data, labels, batch_size, shuffle=True )

In [ ]:
tf.reset_default_graph()

sess = tf.Session( )

num_epochs  = 10000
batch_size  = 32
data_dir    = "./data/partial/cifar-010-bin/"
code_length = 100;

with tf.device( '/cpu:0' ):
    data_batch, labels_batch = creader( data_dir, batch_size, num_epochs )
    
    # filename_queue = tf.train.string_input_producer( ["./data/partial/cifar-010-bin/data_batch_1.bin"] )
    # 
    # reader = tf.FixedLengthRecordReader( record_bytes=( 3073 ) )
    # _, value = reader.read( filename_queue )
    # 
    # data = tf.decode_raw( value, tf.uint8 )

data_flat = tf.reshape( data_batch, [-1,32*32*3] )

inputs = data_flat
inputs_shape = inputs.get_shape().as_list()[1:]


tf.train.start_queue_runners(sess)

tmp = sess.run( [data_flat] )

print( tmp )


ERROR:tensorflow:Exception in QueueRunner: Attempting to use uninitialized value read/input_producer/limit_epochs/epochs
	 [[Node: read/input_producer/limit_epochs/CountUpTo = CountUpTo[T=DT_INT64, limit=10000, _device="/job:localhost/replica:0/task:0/device:CPU:0"](read/input_producer/limit_epochs/epochs)]]


Exception in thread QueueRunnerThread-read/input_producer-read/input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 252, in _run
    enqueue_callable()
  File "/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1205, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1350, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.FailedPreconditionError: Attempting to use uninitialized value read/input_producer/limit_epochs/epochs
	 [[Node: read/input_produc

In [ ]:
is_training = True

graph = tf.Graph()
with graph.as_default():
    
    # Global step
    global_step = tf.Variable( 0, trainable=False )
    
    # Input batch
    data_batch = tf.placeholder( tf.float32, shape=[batch_size,32,32,3] )
    
    # Conv layer 1
    conv1 = tf.layers.conv2d( inputs=data_batch, 
                              filters=32, 
                              kernel_size=[3,3],
                              padding='same', 
                              kernel_initializer=tf.contrib.layers.xavier_initializer(),
                              activation=tf.nn.relu
                            )
    
    conv_output = tf.contrib.layers.flatten( tf.layers.batch_normalization( conv1, training=is_train ) )
    
    code_layer = tf.layers.dense( inputs=conv_output, 
                                  units=code_length, 
                                  activation=tf.nn.relu
                                )
    
    code = tf.layers.batch_normalization( code_layer, training=is_training )
    
    code_output = tf.layers.dense( inputs=code, 
                                   units=32*32*3,
                                   activation=tf.nn.relu
                                 )
    
    tconv_input = tf.reshape( code_output, [batch_size,32,32,3] )
    
    tconv1 = tf.layers.conv2d_transpose( inputs=tconv_input,
                                         filters=3, 
                                         padding='same', 
                                         kernel_size=[3,3], 
                                         kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                                         activation=tf.nn.sigmoid
                                       )
    
    loss = tf.nn.l2_loss( data_batch - tconv1 )
    
    learning_rate = tf.train.exponential_decay( learning_rate=1e-4, 
                                                global_step=global_step, 
                                                decay_steps=10000, 
                                                decay_rate=0.95, 
                                                staircase=True
                                              )
    
    trainer = tf.train.AdamOptimizer( learning_rate )
    training_step = trainer.minimize( loss )

In [ ]:
with tf.Session() as sess:
    sess.run( [tf.local_variables_initializer(), tf.global_variables_initializer()] )
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners( sess=sess, coord=coord )
    
    try:
        while not coord.should_stop():
            _, loss_step = sess.run( [training_step, loss], {data_batch: data_batch} )
    finally:
        coord.request_stop()
        coord.join( threads )

In [ ]:
import numpy as np

print( tmp )
print( tf.shape( tconv1 ) )
print( tf.shape( data_batch ) )

In [ ]:
err+1

data_rand = tf.random_normal(
    [2,100],
    mean=0.0,
    stddev=1.0,
    dtype=tf.float32,
    seed=None,
    name=None
)

likeli, _ = tan_nll( data_rand )

sess.run(tf.global_variables_initializer())

tmp, tmp_nll, samp = sess.run( [data_rand, likeli, _] )


# with sess:
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
#     
#     tmp = data_batch.eval( )
#     # print( likeli.eval( ) )
#     
#     coord.request_stop()
#     coord.join( threads )

# print( value.eval() )
# value = sess.run( [value] )



In [ ]:
# Pass it through the tan
# print( tmp )

import numpy as np
print( np.shape( tmp ) )
print( np.shape( tmp_nll ) )

print( tmp_nll )

print( np.shape( samp ) )


# Use some optimizer


# Run the session


# Test how well we performed

In [ ]:
# Generate a LL bound (should use some held-out, non-test data)
#   May need to modify the subsampling process to produce this bounding set

# Run the (full) test set through the trained model


# Plot the average likelihood per class
#   We want to see high likelihoods for cats and dogs and low likelihoods on everything else